In [3]:
import numpy as np
import matplotlib.pyplot as plt
import jieba
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from tqdm import tqdm  #显示进度的库
from pathlib import Path  #
from opencc import OpenCC
import os
import re
import pickle

###wiki词的文件位置
zhwiki = Path("/Users/jv/PycharmProjects/nlp-assignment4/wikiextractor/zhwiki")  ###用 Path 指代zhwiki的路径

from langconv import *

### 将繁体转化为简体

# def tra2sim(input_dir,output_dir):
#     input_dir = zhwiki
#     output_dir = input_dir.parent/ output_dir ##建立输出路径
#     if not output_dir.exists(): output_dir.mkdir(parents=True,exist_ok=True)
#     for wiki in tqdm(input_dir.iterdir()):
#         print(wiki,'正在读入')
#         for wiki_1 in tqdm(wiki.iterdir()):
#             print(wiki_1,'正在翻译')
#             os.system('python -m opencc -c t2s -i {wiki} -o {output}'.format(
#             wiki=str(wiki_1), output=str(output_dir / ('simplified_' + wiki.name+wiki_1.name))
#             ))
###这里用到的是用opencc 将繁体转化为简体
#tra2sim(zhwiki, 'simplified_zhwiki')

###导入停用词
stop_words_file = zhwiki.parent / 'baidu_stop'
stop_words_set = set()
with stop_words_file.open(encoding='utf8') as f:
    for word in f.readlines():
        stop_words_set.add(word.strip('\n'))

print(stop_words_set)
print(len(stop_words_set))
#
simplified_zhwiki=zhwiki.parent / 'simplified_zhwiki'
print(simplified_zhwiki.exists())

def generate_texts(simplified_zhwiki):
    texts = []
    for wiki in tqdm(simplified_zhwiki.iterdir()):
        print(wiki,'正在处理')
        with wiki.open(encoding='utf8') as f:
            for line in f.readlines():
                line = line.strip()
                if line == '' or line.startswith('<doc') or line.startswith('</doc'): continue ##如果该行是空的或者是以doc 开头的都不要
                words = []
                for word in jieba.cut(line): #用结巴分词
                    if word in stop_words_set: continue #如果是停用词的话不要
                    words.append(word) #每一个line都有一个list
                if len(words) > 1: texts.append(words.copy()) #word作为一个list 然后每个文本分开
    return texts
#
texts = generate_texts(simplified_zhwiki)

# print(len(texts))
# #将得到的texts永久保存
with open('corpus.pickle', 'wb') as f:
    pickle.dump(texts, f)

ModuleNotFoundError: No module named 'opencc'

In [ ]:
import pickle
from gensim.models import word2vec
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']


from gensim.models import word2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

with open('/Users/jv/PycharmProjects/nlp-assignment4/corpus.pickle', 'rb') as f:
    corpus = pickle.load(f)
#
#
#
model = word2vec.Word2Vec(corpus, size=300, window=20, min_count=200, workers=4)
print(model.wv['中国'])
#
#
# print(model.most_similar('游戏'))


# def tsne_plot(model):
#     "Creates and TSNE model and plots it"
#     labels = []
#     tokens = []
#
#     for word in model.wv.vocab:
#         tokens.append(model[word])
#         labels.append(word)
#
#     tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
#     new_values = tsne_model.fit_transform(tokens)
#
#     x = []
#     y = []
#     for value in new_values:
#         x.append(value[0])
#         y.append(value[1])
#
#     plt.figure(figsize=(16, 16))
#     for i in range(len(x)):
#         plt.scatter(x[i], y[i])
#         plt.annotate(labels[i],
#                      xy=(x[i], y[i]),
#                      xytext=(5, 2),
#                      textcoords='offset points',
#                      ha='right',
#                      va='bottom')
#     plt.show()
# tsne_plot(model)


def tsne_plot(model):
    labels = []
    tokens = []

    words = ['开心', '难过', '美丽', '丑陋']
    for word in words:
        labels.append(word)
        tokens.append(model[word])
        for sim, _ in model.wv.most_similar(word, topn=50):
            labels.append(sim)
            tokens.append(model[sim])

    tsne = TSNE(perplexity=40, init='pca', n_iter=3500, random_state=42)
    new_vectors = tsne.fit_transform(tokens)

    x, y = [], []
    for v1, v2 in new_vectors:
        x.append(v1)
        y.append(v2)

    plt.figure(figsize=(16, 16))
    for i in range(len(x)):
        plt.scatter(x[i], y[i])
        plt.annotate(labels[i], xy=(x[i], y[i]), xytext=(3, 2), textcoords='offset points', ha='center', va='bottom')

    plt.show()

tsne_plot(model)

In [4]:
%matplotlib inline